### **Atividade Complementar de Bioinformática**
#### André Nakano Miyake - Caso 2

Comecei configurando o ambiente, instalando todas as bibliotecas necessárias num ambiente WSL Ubuntu. 
> Descrição: Paciente lactente bebê, sexo masculino, 6 meses, apresentada na consulta de pediatria geneticista devido a triagem neonatal alterada evidenciando níveis elevados de fenilalanina e atraso incipiente no desenvolvimento neuropsicomotor.

> Procedimento: Painel genético específico para análise de variantes no gene PAH, realizado a partir de amostra de sangue.

> Objetivo: Identificar variantes genéticas responsáveis pela fenilcetonúria (PKU), fornecendo base para diagnóstico precoce e orientações terapêuticas individualizadas para prevenção de complicações neurológicas.


#### 1. Controle de qualidade

In [ ]:
%%bash
# Gerando o relatório da qualidade dos reads de sequenciamento
fastqc -o ../fastqc/ ../raw/caso2-sindrome-metabolica_R1.fastq.gz ../raw/caso2-sindrome-metabolica_R2.fastq.gz

In [ ]:
%%bash
# Filtrar sequências muito pequenas (-m 90) e cortar as pontas
cutadapt \
  -u 5 -U 5 \
  -u -5 -U -5 \
  -m 90 \
  -o ../cutadapt/caso2-sindrome-metabolica-trimmed_R1.fastq.gz \
  -p ../cutadapt/caso2-sindrome-metabolica-trimmed_R2.fastq.gz \
  ../raw/caso2-sindrome-metabolica_R1.fastq.gz \
  ../raw/caso2-sindrome-metabolica_R2.fastq.gz

In [ ]:
%%bash
# Gerando o relatório da qualidade dos reads de sequenciamento, novamente, após o corte
fastqc -o ../fastqc/ ../cutadapt/caso2-sindrome-metabolica-trimmed_R1.fastq.gz ../cutadapt/caso2-sindrome-metabolica-trimmed_R2.fastq.gz

Ao analisar os HTMLs, notei que a qualidade ainda estava ruim, mas decidi prosseguir com os arquivos que tenho para seguir com o fluxo da atividade. A primeira imagem da esquerda é o original sem cutadapt, e a segunda da direita é a após o corte do cutadapt.

<img src="download.png" width="500" alt="Not trimmed"> <img src="download-1.png" width="500" alt="Trimmed"> 


#### 2. Mapeamento com a referência do genoma humano

In [ ]:
%%bash
# Indexando o genoma de referência (Chr12) para o BWA e Samtools
samtools faidx ../reference/Chr12-reference.fasta
bwa index ../reference/Chr12-reference.fasta

In [ ]:
%%bash
# O 'bwa mem' lê os arquivos .gz e a referência, e 'samtools sort' recebe a saída de arquivo sam e cria o arquivo .bam
bwa mem ../reference/Chr12-reference.fasta \
  -R '@RG\tID:caso2\tSM:caso2\tPL:ILLUMINA\tLB:lib1\tPU:unit1' \
  ../cutadapt/caso2-sindrome-metabolica-trimmed_R1.fastq.gz \
  ../cutadapt/caso2-sindrome-metabolica-trimmed_R2.fastq.gz \
  | samtools sort -o ../bwa/caso2-sindrome-metabolica-mapped-host.bam

In [ ]:
%%bash
# Cria um arquivo .bai na mesma pasta do BAM (../bwa/)
samtools index ../bwa/caso2-sindrome-metabolica-mapped-host.bam

Abrindo o arquivos no IGV, foi possível identificar gaps, mismatches e mutações. No entanto, houveram apenas duas mutações significativas que ocorreram em éxons. A localização dessas bases mutadas são '102852923' e '102894918'. A primeira é uma base A com vários missmatches com uma base G da leitura.

<img src="image (1).png" width="500"> <img src="image.png" width="500">

#### 3. Chamada de variantes

In [ ]:
%%bash
# Indexar a referência e BAM para chamada de variantes com GATK, o samtools dict cria o arquivo .dict que o GATK precisa
samtools dict ../reference/Chr12-reference.fasta > ../reference/Chr12-reference.dict
samtools index ../bwa/caso2-sindrome-metabolica-mapped-host.bam

In [ ]:
%%bash
# Chamada de variantes com GATK
gatk HaplotypeCaller \
  -R ../reference/Chr12-reference.fasta \
  -I ../bwa/caso2-sindrome-metabolica-mapped-host.bam \
  -O ../gatk/caso2-sindrome-metabolica-host-variants.vcf

Analisando o arquivo .vcf, os alelos de interesse são os '102852923' e '102894918', previamente identificados. As frequências alélicas de ambos é de 0.5 , isso significa que são variantes heterozigóticas, e portanto a mutação é de alelo dominante (já que é expressa mesmo em heterozigose).

#### 4. Anotação de variantes

In [ ]:
%%bash
# script oficial baixa cache do genoma humano (GRCh38)
vep_install -a cf -s homo_sapiens -y GRCh38 --NO_UPDATE

In [ ]:
%%bash
# Index arquivos vcf para anotação
bgzip ../gatk/caso2-sindrome-metabolica-host-variants.vcf
tabix -p vcf ../gatk/caso2-sindrome-metabolica-host-variants.vcf.gz

In [ ]:
%%bash
# Executa a anotação VEP
vep -i ../gatk/caso2-sindrome-metabolica-host-variants.vcf.gz \
    --output_file ../vep/caso2-sindrome-metabolica-host-variants.vep.tsv \
    --tab --force_overwrite \
    --offline --cache --dir_cache ~/.vep \
    --assembly GRCh38 \
    --symbol --hgvs --canonical --af_1kg --max_af \
    --everything --pick \
    --stats_file ../vep/caso2-sindrome-metabolica-host-variants.vep.html

Analisando o arquivo .tsv, observamos com mais atenção as linhas 150 e 152 (localização '102852923' e '102894918', respectivamente). O primeiro tem predição de impacto provavelmente danosa (1), e tem significância clínica patogênica. O segundo tem predição de impacto benigna (0,003), e tem significância clínica de classificação incerta (VUS).